In [1]:
import pandas as pd
import numpy as np
import glob
from sklearn.model_selection import StratifiedKFold
from sklearn.externals import joblib
import lightgbm as lgb
import gc
import seaborn as sns
import matplotlib.pyplot as plt
import sys

import warnings
warnings.filterwarnings("ignore")

In [92]:
train = pd.read_csv('../data/Processed/train.csv')
X_train = train.loc[:,[c for c in train.columns if c != 'patient_id']]
y_train = pd.read_csv('../data/raw/train_labels.csv', usecols=['heart_disease_present'])
test = pd.read_csv('../data/Processed/test.csv')
X_test = test.loc[:,[c for c in test.columns if c != 'patient_id']]

In [93]:
train.head()

,patient_id,slope_of_peak_exercise_st_segment,thal,resting_blood_pressure,chest_pain_type,num_major_vessels,fasting_blood_sugar_gt_120_mg_per_dl,resting_ekg_results,serum_cholesterol_mg_per_dl,oldpeak_eq_st_depression,sex,age,max_heart_rate_achieved,exercise_induced_angina
0,0z64un,1,1,128,2,0,0,2,308,0.0,1,45,170,0
1,ryoo3j,2,1,110,3,0,0,0,214,1.6,0,54,158,0
2,yt1s1x,1,1,125,4,3,0,2,304,0.0,1,77,162,1
3,l2xjde,1,2,152,4,0,0,0,223,0.0,1,40,181,0
4,oyt4ek,3,2,178,1,0,0,2,270,4.2,1,59,145,0


In [94]:
model_name = 'lgbm_3'


params = {'max_depth':2,
          'metric':'binary_logloss',
          'num_threads': 1,
          'max_delta_step': 0.2,
          'n_estimators':10000,
          'learning_rate':0.05,
          'colsample_bytree':1,
          'objective':'binary',
          'n_jobs':-1,
          'seed':42,
          'bagging_fraction':0.8,
          'lambda_l1':0,
          'lambda_l2':0,
          'max_bin': 10}

# Stacking con Folds anidados 5 a 5

In [110]:
lgb_model = lgb.LGBMClassifier(**params)

train_ids = X_train.index
k1 = 7
k2 = 5
skf1 = StratifiedKFold(n_splits=k1, shuffle=True, random_state=42)
skf2 = StratifiedKFold(n_splits=k2, shuffle=True, random_state=42)

In [111]:
len(X_fit_ids), len(y_fit)

NameError: name 'y_fit' is not defined

In [112]:
counter1 = 1
y_preds = np.zeros(X_test.shape[0])
be = 0
stack = pd.DataFrame()
stack2 = pd.DataFrame()
for train_index, test_index in skf1.split(train_ids, y_train):
    counter2 = 1
    print('Fold k1 {}\n'.format(counter1))

    X_fit, X_val = X_train.iloc[train_index, :], X_train.iloc[test_index, :]
    y_fit, y_val = y_train.iloc[train_index], y_train.iloc[test_index]
    
    X_fit_ids = X_fit.index
    
    y_preds_stack = np.zeros(X_val.shape[0])
    
    for train_index2, test_index2 in skf2.split(X_fit_ids, y_fit):
        
        print('Fold k2 {}\n'.format(counter2))
        
        X_fit2, X_val2 = X_fit.iloc[train_index2, :], X_fit.iloc[test_index2, :]
        y_fit2, y_val2 = y_fit.iloc[train_index2], y_fit.iloc[test_index2]

        lgb_model.fit(X_fit2,
                      y_fit2,
                      eval_set=[(X_val2, y_val2)],
                      verbose=400,
                      early_stopping_rounds=5)

        y_preds += lgb_model.predict_proba(X_test)[:,1]
        y_preds_stack += lgb_model.predict_proba(X_val)[:,1]

        be += lgb_model.best_score_['valid_0']['binary_logloss']
        
        del X_fit2
        del X_val2
        del y_fit2
        del y_val2
        del train_index2
        del test_index2
        gc.collect()

    #     print('Guardamos el modelo')
    #     joblib.dump(lgb_model, '../saved_models/{}_{}.pkl'.format(model_name, counter))

    #     ft_importances += lgb_model.feature_importances_

        counter2 += 1
        
    stack2['patient_id'] = train['patient_id'][test_index].tolist()
    stack2['heart_disease_present'] = y_preds_stack/k2
    stack = pd.concat([stack, stack2], axis = 0)
    
    del X_fit
    del X_val
    del y_fit
    del y_val
    del train_index
    del test_index
    gc.collect()

    counter1 += 1
    
y_preds = y_preds / (k1*k2)

print('\n\nBEST SCORE MEAN:', be / (k1*k2))


Fold k1 1

Fold k2 1

Training until validation scores don't improve for 5 rounds.
Early stopping, best iteration is:
[224]	valid_0's binary_logloss: 0.491587
Fold k2 2

Training until validation scores don't improve for 5 rounds.
Early stopping, best iteration is:
[250]	valid_0's binary_logloss: 0.450638
Fold k2 3

Training until validation scores don't improve for 5 rounds.
Early stopping, best iteration is:
[223]	valid_0's binary_logloss: 0.547072
Fold k2 4

Training until validation scores don't improve for 5 rounds.
Early stopping, best iteration is:
[265]	valid_0's binary_logloss: 0.364243
Fold k2 5

Training until validation scores don't improve for 5 rounds.
Early stopping, best iteration is:
[132]	valid_0's binary_logloss: 0.57792
Fold k1 2

Fold k2 1

Training until validation scores don't improve for 5 rounds.
Early stopping, best iteration is:
[209]	valid_0's binary_logloss: 0.463821
Fold k2 2

Training until validation scores don't improve for 5 rounds.
Early stopping, bes

ValueError: Length of values does not match length of index

In [113]:
stack.shape

(54, 2)

In [114]:
stack.head()

,patient_id,heart_disease_present
0,0z64un,0.164153
1,yt1s1x,0.620082
2,3nwy2n,0.810374
3,cvux3j,0.308505
4,lek9q9,0.202178


In [9]:
stack.to_csv('../stacking/data/train_{}.csv'.format(model_name), index=False)

In [115]:
y_preds[:5]

array([ 7.15281591,  4.25085161, 12.19486691,  4.0201705 , 11.5755431 ])

In [11]:
sub = pd.DataFrame({'patient_id': test['patient_id'], 'heart_disease_present': y_preds})

In [12]:
sub.to_csv('../stacking/data/test_{}.csv'.format(model_name), index=False)